In this notebook we:

2. Generate a Dilithium keypair, signature and verification using `dilithium-py`
3. Compare Dilithium against RSA and ECDSA at the NIST-2, NIST-3, and NIST-5 security levels
4. Summarize the key and signature sizes and observe computational time

## Setup and helper functions

In [ ]:
import time

from cryptography.hazmat.primitives.asymmetric import rsa, ec, padding
from cryptography.hazmat.primitives import hashes
from dilithium_py.dilithium import Dilithium2, Dilithium3, Dilithium5

msg = b"A test message"

def print_row(name, pub, priv, sig):
    print(f"{name:<18} {pub:>12} {priv:>13} {sig:>15}")

results = {}


## NIST Level 2 (192 bits)

### ECDSA P-256

In [61]:
print("\nNIST Level 2 — ECDSA P-256")

start = time.time()
priv = ec.generate_private_key(ec.SECP256R1())
elapsed = time.time() - start
print(f"Key generation time: {elapsed:.4f} seconds")

sig = priv.sign(msg, ec.ECDSA(hashes.SHA256()))

print_row("ECDSA P-256", 64, 32, len(sig))
results["NIST2_ECDSA"] = ("ECDSA P-256", 64, 32, len(sig))




NIST Level 2 — ECDSA P-256
Key generation time: 0.0025 seconds
ECDSA P-256                  64            32              72


### RSA-3072

In [37]:
print("\nNIST Level 2 — RSA-3072")

start = time.time()

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=3072
)

elapsed = time.time() - start
print(f"Key generation time: {elapsed:.2f} seconds")

public_key = private_key.public_key()

signature = private_key.sign(
    msg,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

public_key.verify(
    signature,
    msg,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

size = private_key.key_size // 8
print_row("RSA-3072", size, size, len(signature))
results["NIST2_RSA"] = ("RSA-3072", size, size, len(signature))



NIST Level 2 — RSA-3072
Key generation time: 0.11 seconds
RSA-3072                    384           384             384


## Dilithium2

In [50]:
print("\nNIST Level 2 — Dilithium2")

start = time.time()

pk, sk = Dilithium2.keygen()

elapsed = time.time() - start
print(f"Key generation time: {elapsed:.4f} seconds")

sig = Dilithium2.sign(sk, msg)
Dilithium2.verify(pk, msg, sig)

print_row("Dilithium2", len(pk), len(sk), len(sig))
results["NIST2_Dilithium"] = ("Dilithium2", len(pk), len(sk), len(sig))


NIST Level 2 — Dilithium2
Key generation time: 0.0082 seconds
Dilithium2                 1312          2528            2420


NIST Level 3

In [59]:
print("\nNIST Level 3 — ECDSA P-384")

start = time.time()
priv = ec.generate_private_key(ec.SECP384R1())
elapsed = time.time() - start
print(f"Key generation time: {elapsed:.4f} seconds")

sig = priv.sign(msg, ec.ECDSA(hashes.SHA256()))

print_row("ECDSA P-384", 96, 48, len(sig))
results["NIST3_ECDSA"] = ("ECDSA P-384", 96, 48, len(sig))



NIST Level 3 — ECDSA P-384
Key generation time: 0.0012 seconds
ECDSA P-384                  96            48             103


In [40]:
print("\nNIST Level 3 — RSA-7680")

start = time.time()

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=7680
)

elapsed = time.time() - start
print(f"Key generation time: {elapsed:.2f} seconds")

public_key = private_key.public_key()

signature = private_key.sign(
    msg,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

size = private_key.key_size // 8
print_row("RSA-7680", size, size, len(signature))
results["NIST3_RSA"] = ("RSA-7680", size, size, len(signature))



NIST Level 3 — RSA-7680
Key generation time: 3.68 seconds
RSA-7680                    960           960             960


In [51]:
print("\nNIST Level 3 — Dilithium3")

start = time.time()
pk, sk = Dilithium3.keygen()
elapsed = time.time() - start
print(f"Key generation time: {elapsed:.4f} seconds")

sig = Dilithium3.sign(sk, msg)
print_row("Dilithium3", len(pk), len(sk), len(sig))
results["NIST3_Dilithium"] = ("Dilithium3", len(pk), len(sk), len(sig))


NIST Level 3 — Dilithium3
Key generation time: 0.0389 seconds
Dilithium3                 1952          4000            3293


NIST Level 5

In [60]:
print("\nNIST Level 5 — ECDSA P-521")

start = time.time()
priv = ec.generate_private_key(ec.SECP521R1())
elapsed = time.time() - start
print(f"Key generation time: {elapsed:.4f} seconds")

sig = priv.sign(msg, ec.ECDSA(hashes.SHA256()))

print_row("ECDSA P-521", 132, 66, len(sig))
results["NIST5_ECDSA"] = ("ECDSA P-521", 132, 66, len(sig))


NIST Level 5 — ECDSA P-521
Key generation time: 0.0026 seconds
ECDSA P-521                 132            66             139


In [55]:
print("\nNIST Level 5 — RSA-15360")

start = time.time()

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=15360
)

elapsed = time.time() - start
print(f"Key generation time: {elapsed:.2f} seconds")

size = private_key.key_size // 8
signature = private_key.sign(
    msg,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

print_row("RSA-15360", size, size, len(signature))
results["NIST5_RSA"] = ("RSA-15360", size, size, len(signature))


NIST Level 5 — RSA-15360
Key generation time: 17.17 seconds
RSA-15360                  1920          1920            1920


In [56]:
print("\nNIST Level 5 — Dilithium5")

start = time.time()
pk, sk = Dilithium5.keygen()
elapsed = time.time() - start
print(f"Key generation time: {elapsed:.4f} seconds")

sig = Dilithium5.sign(sk, msg)
print_row("Dilithium5", len(pk), len(sk), len(sig))
results["NIST5_Dilithium"] = ("Dilithium5", len(pk), len(sk), len(sig))



NIST Level 5 — Dilithium5
Key generation time: 0.0185 seconds
Dilithium5                 2592          4864            4595


Final Comparison

In [62]:
def print_table(title, keys):
    print(f"\n{title}")
    header = f"{'Algorithm':<18} {'Public (B)':>12} {'Private (B)':>13} {'Signature (B)':>15}"
    print(header)
    print("-" * len(header))
    for k in keys:
        alg, pub, priv, sig = results[k]
        print(f"{alg:<18} {pub:>12} {priv:>13} {sig:>15}")

print_table("NIST Level 2 (~128-bit)",
            ["NIST2_RSA", "NIST2_ECDSA", "NIST2_Dilithium"])

print_table("NIST Level 3 (~192-bit)",
            ["NIST3_RSA", "NIST3_ECDSA", "NIST3_Dilithium"])

print_table("NIST Level 5 (~256-bit)",
            ["NIST5_RSA", "NIST5_ECDSA", "NIST5_Dilithium"])



NIST Level 2 (~128-bit)
Algorithm            Public (B)   Private (B)   Signature (B)
-------------------------------------------------------------
RSA-3072                    384           384             384
ECDSA P-256                  64            32              72
Dilithium2                 1312          2528            2420

NIST Level 3 (~192-bit)
Algorithm            Public (B)   Private (B)   Signature (B)
-------------------------------------------------------------
RSA-7680                    960           960             960
ECDSA P-384                  96            48             103
Dilithium3                 1952          4000            3293

NIST Level 5 (~256-bit)
Algorithm            Public (B)   Private (B)   Signature (B)
-------------------------------------------------------------
RSA-15360                  1920          1920            1920
ECDSA P-521                 132            66             139
Dilithium5                 2592          4864            